In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import csv

import tensorflow as tf
from tensorflow import shape,math
from tensorflow.keras import Input,layers,Model
from tensorflow.keras.losses import mse,binary_crossentropy
from tensorflow.keras.utils import plot_model

import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


latent_dims = 3

Num GPUs Available:  1


In [ ]:
import pandas as pd
import numpy as np

# Create two sample DataFrames
df1 = pd.DataFrame([[1, 2, 3], [4, 5, 6]])
df2 = pd.DataFrame([[7, 8, 9], [10, 11, 12]])

# Concatenate the DataFrames horizontally
combined_df = pd.concat([df1, df2], axis=0)

# Reshape the combined DataFrame into (2, 2, 3)
reshaped_df = np.reshape(combined_df.values, (2, 2, 3))

# Print the reshaped DataFrame
print(reshaped_df.shape)


In [121]:
import os
import pandas as pd

output_dir = os.path.join(".", "output")  # Path to the output directory

# Get a list of subdirectories (timestamped folders) in the output directory
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]

# Initialize an empty DataFrame
combined_df = pd.DataFrame()
print(subdirs)

# Loop through each subdirectory and load the CSV files
for subdir in subdirs:
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")

    # Load the action and obs CSV files
    action_df = pd.read_csv(action_filename)
    obs_df = pd.read_csv(obs_filename)

    # Concatenate the DataFrames horizontally
    combined_data = pd.concat([action_df, obs_df], axis=1)
    # combined_data['FileName'] = os.path.basename(subdir)

    # Append the combined data to the combined DataFrame
    combined_df = pd.concat([combined_df, combined_data], ignore_index=True)



combined_df = np.reshape(combined_df.values, (len(subdirs), 999, 10))


# Print the combined DataFrame
print(combined_df.shape)


['./output/20230714_115330', './output/20230714_115045', './output/20230714_115149']
(3, 999, 10)


In [122]:
combined_df

array([[[ 1.        , -0.4887499 , -0.12518893, ..., -1.48101765,
          3.16040391, -2.97962481],
        [ 1.        ,  0.6673651 , -0.19520388, ..., -1.98274603,
          3.5745104 , -2.21610183],
        [ 1.        ,  1.        , -0.27878554, ..., -2.18284202,
          3.36718039, -0.84459476],
        ...,
        [-1.        , -1.        , -0.26966133, ...,  2.38654608,
         -3.43401684, -0.63860864],
        [-1.        , -1.        , -0.17271106, ...,  2.45474947,
         -3.33365148, -1.16270128],
        [-1.        , -1.        ,  0.07148086, ...,  0.00829224,
         -0.04005762, -0.01546256]],

       [[ 1.        , -0.4887499 , -0.12518893, ..., -1.48101765,
          3.16040391, -2.97962481],
        [ 1.        ,  0.6673651 , -0.19520388, ..., -1.98274603,
          3.5745104 , -2.21610183],
        [ 1.        ,  1.        , -0.27878554, ..., -2.18284202,
          3.36718039, -0.84459476],
        ...,
        [-1.        , -1.        , -0.26966133, ...,  

# VAE code

In [123]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import numpy as np
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

latent_dims = 3

device = torch.device('cpu')

In [131]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 2448)
        
    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((3, 999, 10))


class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(9990,128)
        self.linear2 = nn.Linear(128, latent_dims)
        self.linear3 = nn.Linear(128, latent_dims)
        
        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device) # hack to get sampling on the GPU
        self.N.scale = self.N.scale.to(device)
        self.kl = 0
    
    def forward(self, x):
        x = torch.flatten(x, start_dim=2)
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        
        # Re-paramaterization trick !
        z = mu + sigma*self.N.sample(mu.shape)

        # Kullback-Leibler Divergence
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        
        return z
    
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)
    
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum() + autoencoder.encoder.kl
            loss.backward()
            opt.step()
        print (f'Train epoch:{epoch} with previous loss {loss}');
    return autoencoder

In [132]:
from torchsummary import summary
vae = VariationalAutoencoder(latent_dims).to(device) # GPU
summary(vae, input_size=combined_df.shape)

RuntimeError: shape '[3, 999, 10]' is invalid for input of size 14688

In [128]:
vae = VariationalAutoencoder(latent_dims).to(device) # GPU
vae = train(vae, combined_df)

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [34]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 2448)
        
    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z


class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(11, 5994)
        self.linear2 = nn.Linear(5994, latent_dims)
        self.linear3 = nn.Linear(5994, latent_dims)
        
        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device)  # hack to get sampling on the GPU
        self.N.scale = self.N.scale.to(device)
        self.kl = 0
    
    def forward(self, x):
        x = torch.flatten(x, start_dim=2)
        x = F.relu(self.linear1(x))
        mu = self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        
        # Re-parameterization trick!
        z = mu + sigma * self.N.sample(mu.shape)

        # Kullback-Leibler Divergence
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        
        return z.reshape((-1, latent_dims))
    
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)
    
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


from torchsummary import summary
vae = VariationalAutoencoder(latent_dims).to(device) # GPU
summary(vae, input_size=combined_df.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1           [-1, 2997, 5994]          71,928
            Linear-2              [-1, 2997, 5]          29,975
            Linear-3              [-1, 2997, 5]          29,975
VariationalEncoder-4                    [-1, 5]               0
            Linear-5                  [-1, 512]           3,072
            Linear-6                 [-1, 2448]       1,255,824
           Decoder-7                 [-1, 2448]               0
Total params: 1,390,774
Trainable params: 1,390,774
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.13
Forward/backward pass size (MB): 137.32
Params size (MB): 5.31
Estimated Total Size (MB): 142.76
----------------------------------------------------------------
